In [1]:
#pip install cvrplib

In [18]:
'''
Программа читает очередной .vrp-файл из папки datafiles,
делает начальное распределение клиентов по маршрутам и в цикле
SEQ_COUNT выполняет 2 вида перестановок клиентов:
 - 1 клиент переносится из одного маршрута в другой;
 - 2 клиента меняются местами в одном и том же или разных маршрутах.

После этого проверяется корректность значения capacity и суммы demands на маршрутах
и проверяется суммарная длина маршрута. При улучшении - происходит переход.
При ухудшении переход совершается с вероятностью -exp((T-x)/kt).
Если происходит улучшение, счётчик цикла по SEQ_COUNT (переменная opt) 
сбрасывается и отсчёт начинается заново.
При отсутствии прогресса после SEQ_COUNT/2 попыток тип изменения меняется на
перестановки клиентов внутри маршрута. По достижении максимума цикл прекращается,
выводятся данные по лучшему распределению и загружается новый файл.
Если суммарное число попыток превысит MAX_COUNT - цикл также прекращается.
'''

from random import randrange, random, seed
from math import exp
import glob, os
import cvrplib
import sys
from datetime import datetime

MAX_COUNT = 100000000  # 100kk
SEQ_COUNT = 2000000    # 2kk
K_T = 0.739

# Считаем суммарную Capacity для маршрута
def calc_cap(route, cd):
    sum = 0
    for i in range(1,len(route)):
        sum = sum + cd[route[i]]
    return sum

# Считаем суммарную длину маршрута
def total_length(cr,var):
    sum = 0
    for r in range(len(cr)): # r - маршрут;
        for i in range(len(cr[r])): # i - клиенты;
            # Сумма евклидовых расстояний всех маршрутов
            sum = sum + var.distances[cr[r][i]][cr[r][i+1]]
            if cr[r][i+1] == 0:
                break
    return sum

# Поиск индекса последнего клиента на маршруте
def last_client(route):
    i = 1 # 0 - депо, эта позиция постоянная и её не надо учитывать
    # Маршруты содержат значения: [0, cl1, cl2, ..., clN, 0, 0, 0, ... 0]
    while route[i] != 0:
        i += 1
    return i

def show_routes(cr,goal,result):
    for r in range(len(cr)): # Вывод результата
        print ("Route #",r+1, sep='', end=': ') 
        for i in range(1,len(cr[r])):
            if cr[r][i] != 0:
                print(cr[r][i], end=' ')
        print()
    if result > goal:
        print( "FAILED: ", )
    print( "Cost", result ) # Длина маршрута передаётся для вывода в виде аргумента, а не вычисляется

def copy_routes(src,dest):
    for r in range(len(src)):
        dest[r] = src[r][:]
    
# Меняем два элемента (клиентов) местами
def cswap(cr,var,kt,T,change):
    # cr - маршруты, var - данные из файла, kt - знаменатель для exp(), T - лучшая длина, change - число перестановок
    # Суммарная длина всех маршрутов
    len1 = total_length( cr, var )
    # Создаём массив для хранения копии на случай отката
    cr2 = [0] * len(cr)
    for i in range(len(cr)):
        cr2[i] = [0] * var.dimension
    # Копируем все маршруты
    copy_routes(cr,cr2)
    ok = 0 # Число состоявшихся замен/перемещений
    while ok != change:
        r1 = randrange(len(cr))
        if change > 3:
            r2 = r1
        else:
            r2 = randrange(len(cr))
        # Последние колонки в маршрутах
        lc1 = last_client(cr[r1])+1 # +1 - для выражения range
        lc2 = last_client(cr[r2])+1
        # Выбираем рандомных клиентов из маршрутов (может быть и в рамках одного маршрута)
        i1 = randrange(1,lc1)
        i2 = randrange(1,lc2)
        while cr[r1][i1] == cr[r2][i2]: # Исключаем обмен на себя
            r1 = randrange(len(cr))
            lc1 = last_client(cr[r1])+1
            i1 = randrange(1,lc1)
        ins = randrange(3)
        if ins > 0: # Обмен
            swap = cr[r1][i1]
            cr[r1][i1] = cr[r2][i2]
            cr[r2][i2] = swap
            while (cr[r1][i1] == 0) & (cr[r1][i1+1] != 0):
                cr[r1][i1] = cr[r1][i1+1]
                cr[r1][i1+1] = 0

                i1 = i1 + 1
            while (cr[r2][i2] == 0) & (cr[r2][i2+1] != 0):
                cr[r2][i2] = cr[r2][i2+1]
                cr[r2][i2+1] = 0
                i2 = i2 + 1
        else: # Вставка
            swap = cr[r1][i1]          
            # 1 2 3 [i1]-> 5 6 7 8 i2 11 0 0...
            # 1 2 3 5 6 7 8 i2 11 0 0 0...
            # 1 2 3 5 6 7 8 i1 i2 11 0 0...
            if (r1 == r2) & (i1 < i2): # Второй клиент временно сместится влево
                i2 -= 1
            cr[r1][i1] = 0
            while (cr[r1][i1] == 0) & (cr[r1][i1+1] != 0):
                cr[r1][i1] = cr[r1][i1+1]
                cr[r1][i1+1] = 0
                i1 += 1
            while swap != 0:
                k = cr[r2][i2]
                cr[r2][i2] = swap
                swap = k
                i2 += 1
        ok += 1
        #if change > 3:
        #    ok = change
    ret = 1 # Изначально считаем, что перестановка удалась
    for r in range(len(cr)): 
        if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, не превысили ли мы вместимость маршрута
            ret = 0
            break
    if ret == 1: 
        len2 = total_length(cr,var) # Вычисляем новый итог
        if (len2 >= len1): # Если длина стала больше, по экспоненте определяем вероятность перестановки
            p = exp((len1-len2)/kt)
            q = random()
            if p <= q:
                ret = 0 # Отмена перестановки
    if ret == 0:
        copy_routes(cr2,cr)
    return ret # 1 - перестановка состоялась, 0 - перестановка отменена

entries = glob.glob('datafiles\\*.vrp')
time1 = datetime.now()
for file in entries:
    var = cvrplib.read(file)
    trucks = int(var.name[var.name.find('-k')+2:])
    fr = open(file)
    skip = fr.readline()    # NAME
    val = fr.readline()     # COMMENT
    data = val.split(" ")
    goal = int(data[len(data)-1][:-2])
    fr.close()

    seed(0)
    cr = [0] * trucks       # Массив с маршрутами
    bestcr = [0] * trucks   # Массив для хранения лучшего решения
    ratio = float(sum(var.demands)/(trucks*var.capacity))
    print()
    print("Name:", var.name, " Trucks:", trucks, " Cap:", var.capacity, " var.dimension:",
          var.dimension, " Goal:", goal, " Sum:", sum(var.demands),
          " Ratio:", ratio, sep='')
    start_time = datetime.now()
    for i in range(trucks):
        cr[i] = [0] * var.dimension
        bestcr[i] = [0] * var.dimension
        
    # В случае неуспеха повторяем попытку с другим начальным распределением
    distr = False
    while distr == False:
        for r in range(trucks): # Обнуляем массив
            for i in range(var.dimension):
                cr[r][i] = 0
        # Случайное распределение клиентов по маршрутам с учетом вместимости
        for i in range(1,var.dimension):
            ok = False # i-тый клиент получил свой маршртут
            r = -1 # Признак того, что маршрут ещё не был присвоен
            '''
            Возможна ситуация, когда очередного клиента невозможно поместить в очередной маршрут.
            В этом случае нужно начать распределение заново.
            '''
            while ok == False:
                if r == -1:
                    r = randrange(trucks)
                    fail = 0
                else:
                    r += 1
                    r = r % trucks
                j = last_client(cr[r])
                cr[r][j] = i
                if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, помещается ли клиент в маршрут
                    cr[r][j] = 0
                    fail += 1
                    if fail > trucks:
                        i = 999
                        break
                else:
                    ok = True
        if i != 999:
            distr = True

    T = total_length( cr, var ) # Начальная температура
    print("T =", T)
    opt = 0 # Дополнительный счётчик, который сбрасывается в 0 при нахождении нового лучшего решения
    # ratio = float(sum(var.demands)/(trucks*var.capacity))
    alpha = K_T / var.dimension / trucks * ratio # коэффициент, зависящий от 3 параметров
    kt = T * alpha 
    for count in range(MAX_COUNT):
        if opt > SEQ_COUNT/2:
            change = 4 # Оптимизация маршрутов
            kt = kt * 0.99 # Понижаем вероятность повышения температуры
        else:
            change = randrange(1,3) # 1 или 2 случайные перестановки
        cswap(cr,var,kt,T/2.0,change)
        len2 = total_length( cr, var )
        if T > len2: # Запоминаем лучшее решение
            T = len2
            copy_routes(cr,bestcr)
            opt = 0
            kt = T * alpha
        if count % 10000 == 0:
            print('\r', len2, T, opt, '              ',end='\r',flush=True)
        opt += 1
        if opt > SEQ_COUNT:
            break
    end_time = datetime.now()
    print('\r', '              ',end='\r',flush=True)
    show_routes(bestcr,goal,T)
    print( "Calculation time:", end_time - start_time )
time2 = datetime.now()
print( "\nTotal time:", time2 - time1 )


Name:A-n32-k5 Trucks:5 Cap:100 var.dimension:32 Goal:784 Sum:410 Ratio:0.82
T = 2142
Route #1: 14 28 11 4 23 3 2 6  
Route #2: 21 31 19 17 13 7 26 
Route #3: 30 16 1 12 
Route #4: 29 18 8 9 22 15 10 25 5 20 
Route #5: 24 27 
Cost 784
Calculation time: 0:03:33.621383

Name:A-n33-k5 Trucks:5 Cap:100 var.dimension:33 Goal:661 Sum:446 Ratio:0.892
T = 2064
Route #1: 18 28 23 22          
Route #2: 10 30 25 27 12 4 
Route #3: 11 31 1 21 14 19 6 24 
Route #4: 20 5 26 7 8 13 32 2 
Route #5: 15 17 9 3 16 29 
Cost 661
Calculation time: 0:02:35.935789

Name:A-n33-k6 Trucks:6 Cap:100 var.dimension:33 Goal:742 Sum:541 Ratio:0.9016666666666666
T = 1611
Route #1: 28 27 30 16 25 32    
Route #2: 10 12 21 
Route #3: 22 26 23 24 31 
Route #4: 7 19 29 11 17 
Route #5: 5 2 20 15 9 3 8 4 
Route #6: 13 6 18 1 14 
Cost 742
Calculation time: 0:03:50.128109

Name:A-n34-k5 Trucks:5 Cap:100 var.dimension:34 Goal:778 Sum:460 Ratio:0.92
T = 2017
Route #1: 26 4 33 16 20        
Route #2: 29 1 27 24 30 5 
Route #3:

Route #1: 43 49 5                
Route #2: 46 64 6 34 47 17 
Route #3: 48 57 13 59 44 56 25 
Route #4: 51 39 7 54 11 23 28 62 
Route #5: 26 31 9 22 14 27 15 
Route #6: 4 3 36 35 37 12 1 33 
Route #7: 55 21 10 8 52 24 19 18 
Route #8: 30 50 16 41 2 38 42 61 45 
Route #9: 29 53 40 58 20 32 
FAILED: 
Cost 1268
Calculation time: 0:05:21.380379

Name:A-n69-k9 Trucks:9 Cap:100 var.dimension:69 Goal:1159 Sum:845 Ratio:0.9388888888888889
T = 3845
Route #1: 29 40 56 39 38 68 58   
Route #2: 43 16 50 32 17 10 37 54 
Route #3: 24 12 22 
Route #4: 27 65 55 60 4 47 14 34 
Route #5: 52 48 1 36 62 
Route #6: 23 35 45 20 41 59 8 
Route #7: 26 5 46 63 11 25 42 57 28 
Route #8: 66 13 44 15 3 6 51 30 9 49 33 2 
Route #9: 7 21 61 64 53 18 31 19 
FAILED: 
Cost 1180
Calculation time: 0:07:48.339174

Name:A-n80-k10 Trucks:10 Cap:100 var.dimension:80 Goal:1763 Sum:942 Ratio:0.942
T = 5320
Route #1: 13 74 60 39 3 77 51    
Route #2: 47 69 65 35 43 16 61 68 8 37 2 
Route #3: 76 32 4 45 22 54 33 19 75 31 
Route

Route #1: 60 3 57 30 61 62       
Route #2: 63 21 25 48 9 
Route #3: 17 38 7 11 14 4 46 
Route #4: 12 45 52 22 54 55 56 15 58 
Route #5: 50 49 19 20 10 41 16 
Route #6: 36 6 34 64 44 42 39 
Route #7: 31 18 26 2 47 33 
Route #8: 13 28 5 
Route #9: 59 23 37 65 32 51 27 43 
Route #10: 53 40 1 35 8 24 29 66 
FAILED: 
Cost 1087
Calculation time: 0:06:25.417385

Name:B-n68-k9 Trucks:9 Cap:100 var.dimension:68 Goal:1272 Sum:837 Ratio:0.93
T = 4046
Route #1: 27 66 13 46 43 65      
Route #2: 67 32 6 58 10 24 28 18 36 42 
Route #3: 29 3 11 23 39 25 63 
Route #4: 49 22 2 8 41 37 
Route #5: 62 14 5 54 61 51 57 20 56 33 
Route #6: 19 50 40 55 53 38 
Route #7: 12 45 44 34 16 30 21 7 
Route #8: 59 48 9 26 52 17 1 15 
Route #9: 64 35 47 60 4 31 
FAILED: 
Cost 1310
Calculation time: 0:08:37.258313

Name:B-n78-k10 Trucks:10 Cap:100 var.dimension:78 Goal:1221 Sum:937 Ratio:0.937
T = 4346
Route #1: 61 13 33 55 35 59 36 14 41 68 10 
Route #2: 37 3 20 54 32 7 57 65 47 
Route #3: 75 76 73 31 46 45 51 71 
Ro